In [1]:
import mysql.connector

In [2]:
mycon=mysql.connector.connect(host="localhost", user="root", password="Bhawna@123", database="sboutique")
print(mycon)
mycur=mycon.cursor()

# To check if a customer of a given ID exists or not

In [3]:
def check():
    qry='select Customer_id from customer;'
    
    
    mycur.execute(qry)
    '''Return a list where each element in the list is a tuple fetched from each record in the table. Each tuple contains a single element as only customer IDs are fetched from Customer_id column of each record'''
    d=mycur.fetchall()
    #To create a list of all customer IDs in the table
    list_of_ids=[]
    for ids in d:
        #A list of all customer IDs in the table
        list_of_ids.append(ids[0])
    return list_of_ids

# To create a new account for the customer

In [4]:
#To create a new account for the customer
def cust_ac():
    ask="Y"
    list_of_ids=check()
    while ask in "yY":
        custid=int(input("Enter your customer id..."))
        #to check if a customer already exists with this ID
        if custid in list_of_ids:
            print("This customer ID already exists...\
Try creating a new one")
        else:
        #Tuple to contain details of the customer    
            c_det=()
            cnam=input("First Name: ")
            clnam=input("Last Name: ")
            cphno=input("Phone Number: ")
            cadrs=input("Your Address: ")
            c_det=(custid, cnam, clnam, cphno, cadrs)
            '''Values inserted in the table and default NULL value are 
                    provided for booked product at the time of creation
                    of customer account'''
            qry="insert into customer values(%s,%s,%s,%s,%s,NULL);"
            
            #value of the fields to be entered with the query
            val=c_det
            
            mycur.execute(qry, val)
            mycon.commit()
            print("Customer details entered")
            ask=input("Do you want to continue (Y/N)")
            if ask not in ("Yy"):
                space()
                break

# Get booked products function

In [5]:
# To select all booked products of
# a given customer from the table
def get_bkd_pro(Customer_id):
    qry = 'select bkd_pro from customer\
where cust_id=%s;'
    mycur.execute(qry, (Customer_id,))
    bp = mycur.fetchone()
    bkd_pro = bp[0]
    return bkd_pro

# Function for Customer Login 

In [8]:
def sign_in():
    try:
        ask = int(input("Enter customer ID to sign in: "))
        # Using check function to check whether this account exists or not
        list_of_ids = check()
        if ask in list_of_ids:
            while True:
                print("""Do you want to:
                          1) View Bookings
                          2) Book a product
                          3) Update Self Details
                          4) Cancel booked products
                          enter 'back' to exit""")
                # Take choice of the customer
                ccc = input("Enter choice: ")
                if ccc == "1":
                    # Get booked product function is used where customer ID
                    # is passed as an argument
                    s = get_Products_bkd(ask)
                    # To check if the column has any value
                    if s is None or s == '':
                        print("You have not booked any products yet")
                    else:
                        '''If more than one product is booked,
                        their IDs are stored as a single value
                        separated by '_', so we have to split the
                        string to print each product ID.'''
                        # d is a list containing product IDs
                        d = s.split("_")
                        print("Booked products:")
                        for bkditem in d:
                            print(bkditem)
                elif ccc == '2':
                    # check if the product to be booked exists or not
                    qry = 'SELECT Prod_id FROM products;'
                    mycur.execute(qry)
                    pro_list = mycur.fetchall()
                    '''Contains a list where each element is a tuple fetched
                    from each record, the tuple contains values in the
                    column named pro_nam from the products table'''

                    # empty list to store product IDs
                    list_of_products = []
                    for i in pro_list:
                        list_of_products.append(i[0])

                    # Take ID and quantity of product to be booked
                    pro_id = input('Enter the product id to book products: ')
                    # To add a booked product in the column, we first
                    # need to check if it already contains a value in it
                    if pro_id in list_of_products:
                        # Customer ID is given as a value along with
                        # a query to fetch the booked product for the given ID
                        qry = 'SELECT Products_bkd FROM customer WHERE Customer_id = %s;'
                        mycur.execute(qry, (ask,))
                        pr = mycur.fetchone()
                        # prl is the value fetched from the table
                        prl = pr[0]
                        # When the column is empty, the new product is stored
                        if prl is None or prl == '':
                            qry = 'UPDATE customer SET Products_bkd = %s WHERE Customer_id = %s;'
                            val = (pro_id + '_', ask)
                            mycur.execute(qry, val)
                            mycon.commit()
                            print('Your product is booked!')
                        else:
                            prl1 = prl + pro_id
                            qry2 = 'UPDATE customer SET Products_bkd = %s WHERE Customer_id = %s;'
                            # val2 is the new value containing all booked products
                            # to be stored in the column
                            val2 = (prl1 + '_', ask)
                            mycur.execute(qry2, val2)
                            mycon.commit()
                            print('Your product is booked!')
                    else:
                        print('This product does not exist. Please enter the correct product ID!')
                elif ccc == '3':
                    qry = 'SELECT Customer_id, Customer_name, Last_name, Ph_no, Adrss FROM customer WHERE Customer_id = %s;'
                    mycur.execute(qry, (ask,))
                    # clist contains a list of all values fetched
                    # in the form of a tuple for this customer ID
                    clist = mycur.fetchone()
                    # list of fields to be updated
                    flds = ['Name', 'Last Name', 'Ph.No', 'Address']
                    dic = {}
                    print("Your existing record is:")
                    # The fetched details are stored in the form of key-value pairs in a dictionary
                    for i in range(4):
                        dic[flds[i]] = clist[i+1]
                        print(i+1, ' ', flds[i], ':', clist[i+1])

                    for i in range(len(clist)):
                        updtc = int(input('Enter the choice to update: '))
                        upval = input('Enter ' + flds[updtc-1] + ': ')
                        # Change the value corresponding to the required field
                        dic[flds[updtc-1]] = upval
                        yn = input("Do you want to update other details? (y/n): ")
                        if yn.lower() == 'n':
                            break

                    qry = 'UPDATE customer SET Customer_name = %s, Last_name = %s, Ph_no = %s, Adrss = %s WHERE Customer_id = %s;'
                    updtl = tuple(dic.values()) + (ask,)
                    # The value to be passed along with the query is a tuple
                    # containing the updated details of the given customer ID
                    val = updtl
                    mycur.execute(qry, val)
                    mycon.commit()
                    print("Your details are updated!")
                elif ccc == '4':
                    try:
                        # To get the existing bookings
                        bkd_pro = get_bkd_pro(ask)
                        print("Your Booking(s):")
                        print(bkd_pro)

                        if bkd_pro is None or bkd_pro == '':
                            print('You have no bookings to cancel.')
                        else:
                            cw = input("To cancel all products, enter 'A'\n"
                                       "OR\n"
                                       "Enter the product code to cancel: ")

                            if cw.lower() == 'a':
                                qry = 'UPDATE customer SET Products_bkd = NULL WHERE Customer_id = %s;'
                                mycur.execute(qry, (ask,))
                                mycon.commit()
                                print('All bookings deleted.')
                            elif cw in bkd_pro:
                                # If more than one product, split them based on '_'
                                # x is a list containing all booked products
                                x = bkd_pro[0:-1].split('_')
                                # Delete the required product ID
                                x.remove(cw)
                                updt_pro = ""
                                # Concatenate each product ID in the list to store in the table
                                for item in x:
                                    updt_pro += item + '_'
                                qry = 'UPDATE customer SET Products_bkd = %s WHERE Customer_id = %s;'
                                val = (updt_pro, ask)
                                mycur.execute(qry, val)
                                mycon.commit()
                                print('Booking Cancelled!')
                    except Exception as e:
                        print('Some problem occurred while updating details... Try again.')
                elif ccc.lower() == 'back':
                    print('Successfully logged out')
                    space()
                    break
        else:
            print('This account does not exist')
    except Exception as e:
        print('Some error occurred... Try Again.')

# View Products function

In [9]:
#To fetch values from all columns of
#product table to get product details
def view_pro():
    qry='select * from products;'
    mycur.execute(qry)
    d=mycur.fetchall()
    #contains list of all records
    dic={}
    # Each record fetched is separated into a key value pair
    # and stored in the dictionary where product ID is the key
    for i in d:
        dic[i[0]]=i[1:]
    print('_'*80)
#Printing the dictionary in the form of a table
    print("{:<17} {:<22} {:<23} {:<19}".format(
          'Product id', 'Product name', 'Price', 'Stock'))
    print('_'*80)
    for k, v in dic.items():
        a,b,c=v
        print("{:<17} {:<22} {:<23} {:<19}".format(k, a, b, c))
    print('_'*80)

# Add Products Function

In [10]:
# To add a new product in Products table
def addpro():
    # Display list of products
    view_pro()
    n=int(input('Enter no of items to insert '))
    for j in range(n):
        # Initialize tuple to store
        # product details.
        t=()
        Prod_num=input("Product No.: ")
        Prod_id=input("Product ID: ")
        Prod_price=input("Price: ")
        Prod_stk=input("Stock: ")
        t=(Prod_num, Prod_id, Prod_price, Prod_stk)
        #using MySql query
        qry='insert into products values(%s,%s,%s,%s);'
        val=t
        mycur.execute(qry, val)
        mycon.commit()
        print("Product Added")

# Delete product function

In [11]:
def delpro():
    view_pro()
    delt=input("Enter ID of the product to be deleted: ")
    qry='delete from the products where Prod_id=%s;'
    mycur.execute(qry,(delt,))
    mycon.commit()
    print("Product is deleted")

# Function for employee login

In [12]:
def emp_sign_in():
    try:
        ask=input("Enter employee ID to sign into your account")
        qry='select Employee_id from employee;'
        mycur.execute(qry)
        d=mycur.fetchall()
        lis=[]
        for i in d:
            lis.append(i[0])
        if ask not in lis:
            print("Enter the correct ID")
        else:
            while True:
                space()
                ccc=input("""1.Update delivered records
                             2.Add a new product
                             3.Delete a product
                             Enter 'Back' to logout: """)
                if ccc=='1':
                    cust_id=input("Enter customer id: ")
                    bkd_pro=get_Products_bkd(cust_id)
                    if bkd_pro is None or bkd_pro=='':
                        print("This customer has no booking")
                    else:
                        print('All booking(s):', bkd_pro)
                        pro_id=input("Enter product code to \ remove the delivered product")
                        if pro_id in bkd_pro:
                            x=(bkd_pro[0:-1]).split('_')
                            x.remove(pro_id)
                            updt_pro=''
                            for i in x:
                                updt_pro=updt_pro+i+'_'
                            qry='update customer set Products_bkd=%s \ where Customer_id=%s;'
                            val=(updt_pro, cust_id)
                            mycur.execute(qry,val)
                            mycon.commit()
                            print('Delivered product is removed \ from the database.')
                        else:
                            print("Enter the correct code")
                elif ccc=='2':
                    addpro()
                elif ccc=='3':
                    delpro()
                elif ccc.lower()=='back':
                    print("Successfully logged out")
                    break
    except Exception:
        print('Give the correct input')

# Add employee function

In [13]:
def addemp():
    qry='select * from employee;'
    mycur.execute(qry)
    emp_list=mycur.fetchall()
    print("List of employees")
    for emp in emp_list:
        print("Emp id:",emp[0], "Name:",emp[1], "Last Name:",emp[2], "Phn no:",emp[3])
    ne=[]
    n=int(input("Enter the no.of employees to add"))
    for i in range(1, n+1):
        t=()
        print("Enter employee id")
        idd=int(input(str(i)+')'))
        print('Name')
        nam=input(str(i)+')')
        print("Last name")
        lnam=input(str(i)+')')
        print("Contact no.")
        conco=int(input(str(i)+')'))
        print("Address")
        adrs=input(str(i)+')')
        t=(idd, nam, lnam, conco, adrs)
        ne=ne+[t,]
    qry='insert into employee values(%s,%s,%s,%s,%s);'
    for i in range(len(ne)):
        val=ne[i]
        mycur.execute(qty,val)
        mycon.commit()
    print("All employees details are added.")
    space()

# Function for employer login

In [14]:
#For employee login
def employer():
    while True:
        print()
        print("""Enter your choice:
                  1.View product details
                  2.Add a new employee
                  Enter back to exit""")
        ccc=input("Enter_")
        if ccc=="1":
            view_pro()
        if ccc=="2":
            addemp()
        if ccc.lower()=="back":
            break
            

# Main Program

In [3]:
print("WELCOME! ")
while True:
    print("""Are you a:
              A.Customer
              B.Employee
              C.Employer
              Enter 0 to exit""")
    try:
        if ch in 'aA':
            print("1.Create an account\n 2.Sign in into existing account")
            choice=input("enter-")
            if choice=='1':
                cust_ac()
            elif choice=='2':
                sign_in()
            else:
                print("Enter correct choice")
        if ch in 'bB':
            emp_sign_in()
        if ch in 'cC':
            employer()
        elif ch_lower()=="e":
            print("Thankyou for visiting!")
            break
    except Exception:
        print("Give the right input")
    space()
import mysql.connector
mycon=mysql.connector.connect(host="localhost", user="root", password="Bhawna@123", database="sboutique")
print(mycon)
mycur=mycon.cursor()
def space():
    for i in range(1):
        print()
#To check the customer id
def check():
    qry='select Customer_id from customer;'
    mycur.execute(qry)
    """a list where each element in the list is a tuple
    fetched from each record in table
    Each tuple contains a single element as only customer IDs are fetched
    from Cust_id column of each record"""
    d=mycur.fetchall()
    #to create a list of all customer IDs in the table
    list_of_ids=[]
    for ids in d:
        #a list of all customer IDs in the table
        list_of_ids.append(ids[0])
    return list_of_ids
#To create a new account of customer
def cust_ac():
    ask='Y'
    list_of_ids=check()
    while ask in 'yY':
        custid=int(input("Enter your customer id...."))
        #To check if the customer id already exist
        if custid in list_of_ids:
            print("This Customer Id already exists....\ Try creating a new one")
        else:
            c_det=()
            cnam=input('First Name: ')
            clnam=input('Last Name: ')
            cphno=input('Phone number: ')
            cadrs=input('Your address: ')
            c_det=(custid, cnam, clnam, cphno, cadrs)
            """Values inserted in the table and default NULL value are 
            provided for booked product at the time of creation
            of customer account"""
            qry='insert into customer value(%s,%s,%s,%s,%s,NULL);'
            #value of the fields to be entered in the query
            val=c_det
            mycur.execute(qry, val)
            mycon.commit()
            print("Customer details entered")
            ask=input("Do you want to continue (Y/N)")
            if ask not in ('Yy'):
                space()
                break
#To select all booked products of a given computer from the table 
def get_Products_bkd(Customer_id):
    qry='select Products_bkd from customer where Customer_id=%s;'
    mycur.execute(qry, (Customer_id))
    bp=mycur.fetchone()
    Products_bkd=bp[0]
    return Products_bkd

def sign_in():
    try:
        ask=int(input('Enter customer ID to sign in: '))
        list_of_ids=check()
        if ask in list_of_ids:
            while True:
                print("""Do you want to:
                          1.View Bookings
                          2.Book a product
                          3.Update Self Details
                          4.Cancel booked products
                          Enter 'back' to exit""")
                #Take choice of the customer
                ccc=input('Enter choice - ')
                if ccc='1':
                    # Get booked product function is used where cutomer ID
                    # is passed as an argument
                    s=get_Products_bkd(ask)
                    if s is None or s=='':
                        print('You have not booked products yet')
                    else:
                        """If more than one products are booked,
                        their IDs are stored as a single value
                        separated by '_' so we have to split the
                        string to print each product ID."""
                        
                        #d is the list containing product IDs
                        d=s.split('_')
                        print('Booked products')
                        for bkditems in d:
                            print(bkditems)
                if ccc=='2':
                    #check if the product to be booked exists or not
                    qry='select Prod_id from products;'
                    mycur.execute(qry)
                    pro_list=mycur.fetchall()
                    ''' contains a list where each element is a tuple fetched
                    from each record, the tuple contains values in the
                    column named Prod_nam from products table.'''
                    #empty list to store product IDs
                    list_of_products=[]
                    for i in pro_list:
                        list_of_products.append(i[0])
                    #Take ID and quantity of product to be booked
                    pro_id=input('Enter the product id to boom products:')
                    # To add booked product in the column,we first
                    # need to check if it already contains a value in it
                    if pro_id in list_of_products:
                        # Customer ID is given as value along with
                        # query to fetch booked product for the given ID
                        qry='select Products_bkd from customer where Customer_id=%s;'
                        mycur.execute(qry,(ask))
                        pr=mycur.fetchone()
                        #prl is value fetched from table
                        prl=pr[0]
                        #When the column is empty the new product is to stored
                        if prl is None or prl=='':
                            qry='update customer set Products_bkd=%s where Customer_id=%s;'
                            val=(prod_id+'_',ask)
                            mycur.execute(qry,val)
                            mycon.commit()
                            print("Your product is booked !!")
                '''If there already exists a value in bkd_pro column,
                      new value must be concatenated with the existing
                      one and again stored in the table'''
                       else:
                            prl1=prl+pro_id
                            qry2='update customer set Products_bkd=%s where Customer_id=%s;'
                            val2=(prl1+'_',ask)
                            mycur.execute(qry2,val2)
                            mycon.commit()
                            print("Your product is booked !!")
                    else:
                        print("This product does not exist. \ Please write the correct product id")
                    
                if ccc='3':
                    qry='select Customer_id, Customer_name, Last_name, Ph_no, Adrss\from customer where Customer_id=%s'
                    mycur.execute(qry,(ask,))
                    #clist contains list of all values fetched
                    #in the form of a tuple for this customer ID
                    clist=mycur.fetchone()
                    #list of fields to be updated 
                    flds=['Name','Last Name','Ph.no', 'Address']
                    dic={}
                    print("Your existing record is: ")
                    #The fetched details are stored in the form of key value pair in a dictionary
                    for i in range(4):
                        dic[flds[i]]=clist[i+1]
                        print(i+1, '', flds[i],':', clist[i+1])
                    for i in range(len(clist)):
                        updtc=int(input('Enter choice to update'))
                        upval=input('Enter'+flds[updtc-1]+'')
                    #Change the value corresponding to the required field.
                        dic[flds[updtc-1]]=upval
                        yn=input('Do you want to update other details? y or n')
                        if yn in 'Nn':
                            break
                    qry='update customer set Customer_name=%s, Last_name=%s, Ph_no=%s, \ Adrss=%s where Customer_id=%s;'
                    updtl=tuple(dic.values())+(ask,)
                    #The value to be passed along with the query is a tuple containing updated details of the given customer ID
                    val=(updtl)
                    mycur.execute(qry, val)
                    mycon.commit()
                    print('Your details are updated')
                if ccc='4':
                    try:
                        #To get the existing bookings
                        #Booked products in the table
                        bkd_pro=get_Products_bkd(ask)
                        print('Your Booking(s): \n', bkd_pro)
                        if bkd_pro is None or bkd_pro=='':
                            print('You have no bookings to cancel')
                        else:
                            cw=input('To cancel all products; enter A \nOR\ Enter the product code to cancel;')
                            if cw in 'Aa':
                                qry='update customer set Products_bkd=NULL\ where Customer_id=%s'
                                mycur.execute(qry, (ask,))
                                mycon.commit()
                                print('All bookings deleted')
                            elif cw in bkd_pro:
                                #if more than one products entered,
                                #x is a list containing all booked products
                                x=(bkd_pro[0:-1]).split('_')
                                #Delete the required product ID
                                x.remove(cw)
                                updt_pro=''
                                #Again concatenate each product in the list to store in the table
                                for item in x:
                                    updt_pro=updt_pro+item+'_'
                                    qry='update customer set Products_bkd=%s where Customer_id=%s'
                                    val=(updt_pro, ask)
                                    mycur.execute(qry, val)
                                    mycon.commit()
                                    print('Booking Cancelled !')
                    except Exception:
                        print('Some problem in updating details. Try again')
                if ccc.lower()=='back':
                    print("Successfully logged out")
                    space()
                    break
        else:
            print('This account does not exist')    
    except Exception:
        print('Some error occured. Try Again')
        
#To fetch values from all columns of product table to get product details
def view_pro():
    qry='select * from products;'
    mycur.execute(qry)
    d=mycur.fetchall()
    #contains list of all records
    dic={}
    #Each record fetched is separated into a key value pair and stored in the dictionary where product ID is the key
    for i in d:
        dic[i[0]]=i[1:]
    print('_'*80)
#Printing the dictionary in the form of a table
    print("{:<17}{:<22}{:<23}{:<19}".format('Product id', 'Product name', 'Price', 'Stock'))
    print('_'*80)
    for k, v in dic.items():
        a,b,c=v
        print("{:<17}{:<22}{:<23}{:<19}".format(k,a,b,c))
        print('_'*80)
#To add a new product in Products table
def addpro():
    #Display list of products
    view_pro()
    n=int(input("Enter no. of items to insert"))
    for j in range(n):
        #Initialize tuple to store
        #product details
        t=()
        pronum=input("Product No.")
        proid=input("Product ID: ")
        pprice=int(input("Price: "))
        pstk=int(input("Stock: "))
        t=(pronum, proid, pprice, pstk)
        #Using MySql query
        qry='insert into products values(%s,%s,%s,%s);'
        val=t
        mycur.execute(qry,val)
        mycon.commit()
        print("Product Added")
#To delete a product from the table
def delpro():
    view_pro()
    delt=input("Enter ID of product to be deleted")
    qry='delete from products where Prod_id=%s;'
    mycur.execute(qry, (delt,))
    mycon.commit()
    print("Product is deleted")
#For employee login
def emp_sign_in():
    try:
        ask=input("Enter id to sign in to the account:")
        #To check if the employee id exists
        qry='select Employee_id from employee;'
        mycur.execute(qry)
        d=mycur.fetchall()
        lis=[]
        for i in d:
            lis.append[i[0]]
        if ask not in lis:
            print("Enter the correct id")
        else:
            while True:
                space()
                ccc=input("""1.Update delivered records
                             2.Add a new product
                             3.Delete a product
                             Enter 'Back' to logout""")
                if ccc=='1':
                    cust_id=input("Enter customer id")
                    #Check if the customer has bookings or not
                    bkd_pro=get_Products_bkd(cust_id)
                    if bkd_pro is None or bkd_pro=='':
                        print('This customer has no bookings')
                    else:
                        print('All bookings:', bkd_pro)
                        pro_id=input("Enter product code to\ remove the delivered product")
                        #The product IDs are stored in the form of a single value separated value separated by '_'
                        if pro_id in bkd_pro:
                            x=(bkd_pro[0:-1]).split('_')
                            #Returns a list of all booked products, then remove the delivered product from the list
                            x.remove(pro_id)
                            #Concatenate the existing products using '_'
                            updt_pro=''
                            for i in x:
                                updt_pro=updt_pro+i+'_'
                            qry='update customer set Products_bkd=%s\where Customer_id=%s;'
                            val=(updt_pro, cust_id)
                            mycur.execute(qry, val)
                            mycon.commit()
                            print('Delivered product is removed\from the database.')
                        else:
                            print('Enter the correct code')
                elif ccc=='2':
                    addpro()
                elif ccc=='3':
                    delpro()
                elif ccc.lower()=='back':
                    print('Successfully logged out')
                    break
    except Exception:
        print('Give the correct input')
#To add employee details
def addemp():
    qry='select * from employee;'
    mycur.execute(qry)
    emp_list=mycur.fetchall()
    print('List of employees')
    for emp in emp_list:
        print("Emp Id:", emp[0], "Name:", emp[1], "Last name:", emp[2], "Phone no.:", emp[3])
    ne=[]
    n=int(input("Enter the no. of employees to add"))
    for i in range(1, n+1):
        t=()
        print("Enter employee id")
        idd=int(input(str(i)+')'))
        print("Name")
        nam=input(str(i)+')')
        print("Last name")
        lnam=input(str(i)+')')
        print("Contact no.")
        conno=int(input(str(i)+')'))
        print("Address")
        adrs=input(str(i)+')')
        #A tuple containing details of an employee
        t=(idd, nam, lnam, conno, adrs)
        #List containing details of n number of employees to be added 
        ne=ne+[t, ]
    qry='insert into employee values(%s,%s,%s,%s,%s);'
    #A list containing details of each employee in the form of a tuple is to be passed along with the query
    for i in range(len(ne)):
        val=ne[i]
        mycur.execute(qry,val)
        mycon.commit()
    print("All employee details added.")
    space()
#For employer login
def employer():
    while True:
        print()
        print("""Enter your choice:
                  1.View product details
                  2.Add a new employee
                  Enter back to exit""")
        ccc=input("Enter _________")
        if ccc=='1':
            view_pro()
        if ccc=='2':
            addemp()
        if ccc.lower()=="back":
            break
print("WELCOME!")
#Running a infinite loop
print("WELCOME!")
while True:
    print("""Are you a:
           A. Customer
           B. Employee
           C. Employer
           Enter 'e' to exit""")
    ch = input("Enter: ")
    try:
        if ch in 'aA':
            print("1. Create account\n2. Sign in into an existing account")
            choice = input("Enter: ")
            if choice == '1':
                cust_ac()
            elif choice == '2':
                sign_in()
            else:
                print("Enter correct choice")
        if ch in 'bB':
            emp_sign_in()
        if ch in 'cC':
            employer()
        elif ch.lower() == "e":
            print("Thank you for visiting!")
            break
    except Exception:
        print("Give the right input")
    space()

mycon.close()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 159)

In [1]:
import mysql.connector

mycon = mysql.connector.connect(host="localhost", user="root", password="Bhawna@123", database="sboutique")
print(mycon)
mycur = mycon.cursor()


def space():
    print()


def check():
    qry = 'SELECT Customer_id FROM customer;'
    mycur.execute(qry)
    d = mycur.fetchall()
    list_of_ids = [ids[0] for ids in d]
    return list_of_ids


def cust_ac():
    ask = 'Y'
    list_of_ids = check()
    while ask in 'yY':
        custid = int(input("Enter your customer id: "))
        if custid in list_of_ids:
            print("This Customer Id already exists. Try creating a new one.")
        else:
            c_det = ()
            cnam = input('First Name: ')
            clnam = input('Last Name: ')
            cphno = input('Phone number: ')
            cadrs = input('Your address: ')
            c_det = (custid, cnam, clnam, cphno, cadrs)
            qry = 'INSERT INTO customer VALUES(%s,%s,%s,%s,%s,NULL);'
            val = c_det
            mycur.execute(qry, val)
            mycon.commit()
            print("Customer details entered")
            ask = input("Do you want to continue (Y/N): ")
            if ask not in ('Y', 'y'):
                space()
                break


def get_Products_bkd(Customer_id):
    qry = 'SELECT Products_bkd FROM customer WHERE Customer_id=%s;'
    mycur.execute(qry, (Customer_id,))
    bp = mycur.fetchone()
    Products_bkd = bp[0]
    return Products_bkd


def sign_in():
    try:
        ask = int(input('Enter customer ID to sign in: '))
        list_of_ids = check()
        if ask in list_of_ids:
            while True:
                print("""Do you want to:
                          1. View Bookings
                          2. Book a product
                          3. Update Self Details
                          4. Cancel booked products
                          Enter 'back' to exit""")
                ccc = input('Enter choice: ')
                if ccc == '1':
                    s = get_Products_bkd(ask)
                    if s is None or s == '':
                        print('You have not booked any products yet')
                    else:
                        d = s.split('_')
                        print('Booked products:')
                        for bkditems in d:
                            print(bkditems)
                elif ccc == '2':
                    qry = 'SELECT Prod_id FROM products;'
                    mycur.execute(qry)
                    pro_list = mycur.fetchall()
                    list_of_products = [i[0] for i in pro_list]
                    pro_id = input('Enter the product id to book products: ')
                    if pro_id in list_of_products:
                        qry = 'SELECT Products_bkd FROM customer WHERE Customer_id=%s;'
                        mycur.execute(qry, (ask,))
                        pr = mycur.fetchone()
                        prl = pr[0]
                        if prl is None or prl == '':
                            qry = 'UPDATE customer SET Products_bkd=%s WHERE Customer_id=%s;'
                            val = (pro_id + '_', ask)
                            mycur.execute(qry, val)
                            mycon.commit()
                            print("Your product is booked!")
                        else:
                            prl1 = prl + pro_id
                            qry2 = 'UPDATE customer SET Products_bkd=%s WHERE Customer_id=%s;'
                            val2 = (prl1 + '_', ask)
                            mycur.execute(qry2, val2)
                            mycon.commit()
                            print("Your product is booked!")
                    else:
                        print("This product does not exist. Please enter the correct product id.")
                elif ccc == '3':
                    qry = 'SELECT Customer_id, Customer_name, Last_name, Ph_no, Adrss FROM customer WHERE Customer_id=%s'
                    mycur.execute(qry, (ask,))
                    clist = mycur.fetchone()
                    flds = ['Name', 'Last Name', 'Ph.no', 'Address']
                    dic = {}
                    print("Your existing record is:")
                    for i in range(4):
                        dic[flds[i]] = clist[i + 1]
                        print(i + 1, '', flds[i], ':', clist[i + 1])
                    for i in range(len(clist)):
                        updtc = int(input('Enter choice to update: '))
                        upval = input('Enter ' + flds[updtc - 1] + ': ')
                        dic[flds[updtc - 1]] = upval
                        yn = input('Do you want to update other details? (y/n): ')
                        if yn in 'Nn':
                            break
                    qry = 'UPDATE customer SET Customer_name=%s, Last_name=%s, Ph_no=%s, Adrss=%s WHERE Customer_id=%s;'
                    updtl = tuple(dic.values()) + (ask,)
                    val = updtl
                    mycur.execute(qry, val)
                    mycon.commit()
                    print('Your details are updated')
                elif ccc == '4':
                    try:
                        bkd_pro = get_Products_bkd(ask)
                        print('Your Booking(s): \n', bkd_pro)
                        if bkd_pro is None or bkd_pro == '':
                            print('You have no bookings to cancel')
                        else:
                            cw = input('To cancel all products, enter A\nOR\nEnter the product code to cancel: ')
                            if cw in 'Aa':
                                qry = 'UPDATE customer SET Products_bkd=NULL WHERE Customer_id=%s'
                                mycur.execute(qry, (ask,))
                                mycon.commit()
                                print('All bookings deleted')
                            elif cw in bkd_pro:
                                x = (bkd_pro[0:-1]).split('_')
                                x.remove(cw)
                                updt_pro = ''
                                for item in x:
                                    updt_pro = updt_pro + item + '_'
                                qry = 'UPDATE customer SET Products_bkd=%s WHERE Customer_id=%s'
                                val = (updt_pro, ask)
                                mycur.execute(qry, val)
                                mycon.commit()
                                print('Booking Cancelled!')
                    except Exception:
                        print('Some problem in updating details. Try again')
                elif ccc.lower() == 'back':
                    print("Successfully logged out")
                    space()
                    break
        else:
            print('This account does not exist')
    except Exception:
        print('Some error occurred. Try Again')


def view_pro():
    qry = 'SELECT * FROM products;'
    mycur.execute(qry)
    d = mycur.fetchall()
    print('_' * 80)
    print("{:<17}{:<22}{:<23}{:<19}".format('Product id', 'Product name', 'Price', 'Stock'))
    print('_' * 80)
    for row in d:
        print("{:<17}{:<22}{:<23}{:<19}".format(row[0], row[1], row[2], row[3]))
def addpro():
    view_pro()
    n = int(input("Enter no. of items to insert: "))
    for j in range(n):
        t = ()
        pronum = input("Product No.: ")
        proid = input("Product ID: ")
        pprice = int(input("Price: "))
        pstk = int(input("Stock: "))
        t = (pronum, proid, pprice, pstk)
        qry = 'INSERT INTO products VALUES(%s,%s,%s,%s);'
        val = t
        mycur.execute(qry, val)
        mycon.commit()
        print("Product Added")

def delpro():
    view_pro()
    delt = input("Enter ID of product to be deleted: ")
    qry = 'DELETE FROM products WHERE Prod_id=%s;'
    mycur.execute(qry, (delt,))
    mycon.commit()
    print("Product is deleted")


def emp_sign_in():
    try:
        ask = input("Enter id to sign in to the account: ")
        qry = 'SELECT Employee_id FROM employee;'
        mycur.execute(qry)
        d = mycur.fetchall()
        lis = [i[0] for i in d]
        if ask not in lis:
            print("Enter the correct id")
        else:
            while True:
                space()
                ccc = input("""1. Update delivered records
                             2. Add a new product
                             3. Delete a product
                             Enter 'Back' to logout: """)
                if ccc == '1':
                    cust_id = input("Enter customer id: ")
                    bkd_pro = get_Products_bkd(cust_id)
                    if bkd_pro is None or bkd_pro == '':
                        print('This customer has no bookings')
                    else:
                        print('All bookings:', bkd_pro)
                        pro_id = input("Enter product code to remove the delivered product: ")
                        if pro_id in bkd_pro:
                            x = (bkd_pro[0:-1]).split('_')
                            x.remove(pro_id)
                            updt_pro = ''
                            for i in x:
                                updt_pro = updt_pro + i + '_'
                            qry = 'UPDATE customer SET Products_bkd=%s WHERE Customer_id=%s;'
                            val = (updt_pro, cust_id)
                            mycur.execute(qry, val)
                            mycon.commit()
                            print('Delivered product is removed from the database.')
                        else:
                            print('Enter the correct code')
                elif ccc == '2':
                    addpro()
                elif ccc == '3':
                    delpro()
                elif ccc.lower() == 'back':
                    print('Successfully logged out')
                    break
    except Exception:
        print('Give the correct input')


def addemp():
    qry = 'SELECT * FROM employee;'
    mycur.execute(qry)
    emp_list = mycur.fetchall()
    print('List of employees')
    for emp in emp_list:
        print("Emp Id:", emp[0], "Name:", emp[1], "Last name:", emp[2], "Phone no.:", emp[3])
    ne = []
    n = int(input("Enter the no. of employees to add: "))
    for i in range(1, n + 1):
        t = ()
        print("Enter employee id")
        idd = int(input(str(i) + ')'))
        print("Name")
        nam = input(str(i) + ')')
        print("Last name")
        lnam = input(str(i) + ')')
        print("Contact no.")
        conno = int(input(str(i) + ')'))
        print("Address")
        adrs = input(str(i) + ')')
        t = (idd, nam, lnam, conno, adrs)
        ne = ne + [t, ]
    qry = 'INSERT INTO employee VALUES(%s,%s,%s,%s,%s);'
    for i in range(len(ne)):
        val = ne[i]
        mycur.execute(qry, val)
        mycon.commit()
    print("All employee details added.")
    space()


def employer():
    while True:
        print()
        print("""Enter your choice:
               1. View product details
               2. Add a new employee
               Enter 'back' to exit""")
        ccc = input("Enter: ")
        if ccc == '1':
            view_pro()
        if ccc == '2':
            addemp()
        if ccc.lower() == "back":
            break


print("WELCOME!")
while True:
    print("""Are you a:
           A. Customer
           B. Employee
           C. Employer
           Enter 'e' to exit""")
    ch = input("Enter: ")
    try:
        if ch in 'aA':
            print("1. Create account\n2. Sign in into an existing account")
            choice = input("Enter: ")
            if choice == '1':
                cust_ac()
            elif choice == '2':
                sign_in()
            else:
                print("Enter correct choice")
        if ch in 'bB':
            emp_sign_in()
        if ch in 'cC':
            employer()
        elif ch.lower() == "e":
            print("Thank you for visiting!")
            break
    except Exception:
        print("Give the right input")
    space()

mycon.close()


WELCOME!
Are you a:
           A. Customer
           B. Employee
           C. Employer
           Enter 'e' to exit
Enter: Y

Are you a:
           A. Customer
           B. Employee
           C. Employer
           Enter 'e' to exit
Enter: A
1. Create account
2. Sign in into an existing account
Enter: 2
Enter customer ID to sign in: 1
Do you want to:
                          1. View Bookings
                          2. Book a product
                          3. Update Self Details
                          4. Cancel booked products
                          Enter 'back' to exit
Enter choice: 1
You have not booked any products yet
Do you want to:
                          1. View Bookings
                          2. Book a product
                          3. Update Self Details
                          4. Cancel booked products
                          Enter 'back' to exit
Enter choice: 1
You have not booked any products yet
Do you want to:
                          1. View B

In [4]:
!pip install flask

In [5]:
from flask import Flask, render_template, request

In [6]:
app = Flask(__name__)

In [7]:
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/customer', methods=['GET', 'POST'])
def customer():
    if request.method == 'POST':
        # Handle customer-related actions and logic
        return render_template('customer.html')
    else:
        return render_template('customer.html')

@app.route('/employee')
def employee():
    # Handle employee-related actions and logic
    return render_template('employee.html')

@app.route('/employer')
def employer():
    # Handle employer-related actions and logic
    return render_template('employer.html')

In [8]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
